In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import math as mt
import matplotlib.pyplot as plt

2.6.0


In [2]:
a1=45*np.sqrt(5)
a2=24*np.sqrt(806115)
a3=1560*np.sqrt(49407661)

a1=45*np.sqrt(5)

In [3]:
def basis_fct(k,t):
    if k==1:
        return 1+0*t
    if k==2:
        return -1+np.exp(-t)
    if k==3:
        return np.exp(-t)*t
    if k==4:
        return (1/2)*(-2*np.exp(-t)*t+np.exp(-t)*np.power(t,2))
    if k==5:
        return (-6*np.exp(-t)*t-36*np.exp(-t)*np.power(t,2)+np.exp(-t)*np.power(t,3))/a1
    if k==6:
        return (-24*np.exp(-t)*t-192*np.exp(-t)*np.power(t,2)-1440*np.exp(-t)*np.power(t,3)+np.exp(-t)*np.power(t,4))/a2
    if k==7:
        return (-120*np.exp(-t)*t-1200*np.exp(-t)*np.power(t,2)-10800*np.exp(-t)*np.power(t,3)+100800*np.exp(-t)*np.power(t,4)+np.exp(-t)*np.power(t,5))/a3
    else:
        return 0

In [4]:
def evalCurve0(x,tau):
    xSize=x.size
    Y0=np.zeros(tau.size)
    for k in range(1,xSize+1,1):
        Y0+=x[k-1]*basis_fct(k,tau)
    return np.exp(Y0)


# Simulate one Brownian motion from 0 to T. We discretize daily 
T=1/12#one month simulation time horizon 
L=30#Discretization of BM
s = np.arange(0.0, 1/12, 1/(12*L))
Tminuss=T-s
print(Tminuss)
deltaS=1/(12*L)
Ssize=s.size

# [x1,..,x10] starting value x1e1 +...+ x10e10
def evalCurve(x,BMInc,tau):
    sqrtdeltaS=mt.sqrt(deltaS)
    xSize=x.size
    TauSize=tau.size
    Y0=np.zeros(TauSize)
    drift=np.zeros(TauSize)
    noise=np.zeros(TauSize)
    for k in range(1,xSize+1,1):
        Y0+=x[k-1]*basis_fct(k,tau+T)
        for j in range(0,L,1):
            tmp=basis_fct(k,tau+Tminuss[j])
            drift+=tmp*tmp*deltaS
            noise+=tmp*BMInc[k-1,j]*sqrtdeltaS
        #tmp+=x[k]*np.power(t+T,k-1)*mt.exp(-(t+T))
    return np.exp(Y0-(1/2)*drift+noise)#mt.exp(-(1/2)*T+mt.sqrt(T)*BMInc +tmp)

[0.08333333 0.08055556 0.07777778 0.075      0.07222222 0.06944444
 0.06666667 0.06388889 0.06111111 0.05833333 0.05555556 0.05277778
 0.05       0.04722222 0.04444444 0.04166667 0.03888889 0.03611111
 0.03333333 0.03055556 0.02777778 0.025      0.02222222 0.01944444
 0.01666667 0.01388889 0.01111111 0.00833333 0.00555556 0.00277778]


Now calculate the outputs...

In [6]:
#Callpayoff, one month delivery with delivery starting after one month, strike price K. 
#This cell is just for testing
Strike=1.0

taugrid=np.arange(0.0, 1/12, 1/(12*L))

n=1000
TestSize=5


startingValuesTestset=np.random.uniform(low=-0.5, high=0.5, size=[TestSize,7])
IncrementsBM=np.random.normal(0,1,size=[TestSize,n,7,L])

MCprices=np.zeros(TestSize)
for x in range(0,TestSize,1):
  price=0
  for incBM in range(0,n,1):    
    tmpvec=evalCurve(startingValuesTestset[x],IncrementsBM[x,incBM],taugrid)
    tmp=np.sum(tmpvec)*(12/365)
    price+=max(tmp-Strike,0)
  MCprices[x]=price/n

import matplotlib.pyplot as plt



In [7]:
print(MCprices)

[0.05426984 0.07925    0.60753191 0.06271996 0.01345123]


In [8]:
import sys
print(sys.version_info)
!{sys.executable} -m pip install multiprocess

sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)


In [9]:
from numpy.random import SeedSequence, default_rng
ss = SeedSequence(123456)

# Spawn off 32 child SeedSequences to pass to child processes.
child_seeds = ss.spawn(32*30)
streams = [default_rng(s) for s in child_seeds]

In [10]:
def magic_function(f):
    Strike = 1.0
    n=100000
    TestSizePerNode = 10 #312500
    StartingValuesTestsetLocal= f[1].uniform(low=-0.5, high=0.5, size=[TestSizePerNode,7])

    IncrementsBM = f[1].normal(0,1,size=[TestSizePerNode,n,7,L])
    MCprices=np.zeros(TestSizePerNode)
    for x in range(0,TestSizePerNode,1):
        price=0
        for incBM in range(0,n,1):    
            tmpvec=evalCurve(StartingValuesTestsetLocal[x],IncrementsBM[x,incBM],taugrid)
            tmp=np.sum(tmpvec)*(12/365)
            price+=max(tmp-Strike,0)
        MCprices[x]=price/n
    return StartingValuesTestsetLocal, MCprices


def process_frame(f):
    return f[0], magic_function(f)




In [11]:
from tqdm import tqdm
TestSizePerNode=10
from multiprocess import Pool
#from magic_functions import process_frame

import time
tic = time.perf_counter()

max_pool = 32
for k in range(0,30,1):
    print(k)
    frames_list = [[i,streams[i]] for i in range(k*max_pool,(k+1)*max_pool,1)]
    with Pool(max_pool) as p:
        pool_outputs = list(
            tqdm(
                p.imap(process_frame,
                       frames_list),
                total=len(frames_list)
            )
        )    
    
    print(pool_outputs)
    new_dict = dict(pool_outputs)
    TotalTestSize=TestSizePerNode*max_pool
    startingValues=np.zeros((TotalTestSize,7))
    MCprices=np.zeros(TotalTestSize)
    for x in range(0, max_pool, 1):
        startingValues[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][0]
        MCprices[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][1]
    strk="% s" % k
    filename1="startingvaluesMultiVariateTestset100000Sim"+strk
    filename2="outputsMultiVariateTestset100000Sim"+strk
    np.save(filename1, startingValues)
    np.save(filename2, MCprices)
    
    
toc = time.perf_counter()
print(f"Calc time: {toc - tic:0.4f} seconds")

0


100%|██████████| 32/32 [1:26:27<00:00, 162.11s/it]   

[(0, (array([[ 0.43121012, -0.37309724, -0.10956286, -0.09821822,  0.45122483,
        -0.23397051,  0.09587285],
       [-0.30744494, -0.005644  ,  0.11537568,  0.32497989, -0.35600058,
        -0.17957756,  0.15987916],
       [-0.03294064,  0.29596685, -0.19148758, -0.38018853,  0.35292202,
         0.04945706,  0.28613484],
       [ 0.2840016 , -0.47110234, -0.02105036,  0.29769769,  0.44156592,
         0.44693535,  0.16517371],
       [-0.3368543 , -0.1890448 , -0.45578248,  0.33245255, -0.22142145,
        -0.07758436,  0.21381332],
       [ 0.43064403, -0.18977293,  0.25239723, -0.30325985,  0.12217012,
         0.09602311, -0.05179328],
       [-0.3710508 ,  0.39141016,  0.29386522, -0.45519466,  0.26006402,
        -0.41007788,  0.33758092],
       [-0.26049774, -0.27785206,  0.44599005,  0.26166845,  0.09144087,
        -0.35966998,  0.37131766],
       [-0.03155339, -0.3454618 ,  0.38220823,  0.00479562,  0.44934549,
        -0.32619611,  0.32903414],
       [ 0.26331702, -


100%|██████████| 32/32 [1:26:56<00:00, 163.01s/it]   

[(32, (array([[-0.15750004, -0.21533762, -0.15838381,  0.43371178,  0.15913629,
        -0.48368777, -0.19828243],
       [-0.36406902, -0.49387082, -0.11749833,  0.37857877,  0.28189543,
         0.03327796, -0.08561869],
       [-0.28463902, -0.02270979,  0.07431173,  0.20864974,  0.04428286,
         0.06707337, -0.20201202],
       [ 0.00051412, -0.04397454,  0.45375436,  0.37768506, -0.47379722,
        -0.21142631,  0.3157729 ],
       [-0.40952032, -0.34723956, -0.03651872,  0.4211305 , -0.46938541,
        -0.03021371,  0.44809857],
       [ 0.1763289 ,  0.19009669, -0.05219824, -0.17347535,  0.34157421,
         0.05048544,  0.18189241],
       [-0.42676909, -0.22115492, -0.49289726, -0.15176245, -0.47725473,
         0.00662166,  0.15408432],
       [-0.26203422,  0.17321977,  0.1490694 , -0.37557616,  0.08307972,
        -0.30124724,  0.08550331],
       [-0.03046361, -0.3885292 ,  0.28110801, -0.23564775,  0.32081147,
         0.09943657,  0.15832961],
       [ 0.25507068, 

...

[(864, (array([[ 0.21224155, -0.21428637, -0.37664811,  0.07638948,  0.10643531,
        -0.29826422,  0.26015674],
       [ 0.07072995,  0.0270371 , -0.32327289,  0.12348989, -0.41802181,
         0.19458148, -0.4726723 ],
       [-0.43368019,  0.43914182,  0.38583655, -0.35346519, -0.27624897,
        -0.46532925,  0.35841566],
       [ 0.27472147,  0.2475868 ,  0.17069386, -0.11526109, -0.29856373,
        -0.23483628, -0.25137911],
       [-0.09505435,  0.13522705,  0.39777573, -0.25422734, -0.22447524,
        -0.17734057, -0.26464708],
       [-0.20826084, -0.1064084 , -0.1251929 ,  0.36157187, -0.04515785,
         0.42757377, -0.08211404],
       [ 0.46749025, -0.45998704, -0.1678516 ,  0.2097668 ,  0.30303413,
        -0.06882729,  0.13677295],
       [ 0.46691639, -0.17670845,  0.30815031,  0.02014493,  0.11119621,
         0.11642059,  0.218155  ],
       [ 0.34671949, -0.32706426, -0.3195546 ,  0.37990793, -0.31710508,
         0.31976415, -0.10182852],
       [-0.09047828,


100%|██████████| 32/32 [1:26:44<00:00, 162.64s/it]   

[(896, (array([[-0.07126482,  0.3307221 , -0.13668769, -0.112751  ,  0.34057315,
        -0.11541181, -0.40015898],
       [ 0.11282043, -0.30531021,  0.29002103, -0.02688291, -0.30922633,
         0.29368227,  0.26567365],
       [ 0.29919431,  0.38233748,  0.40540792, -0.09855948,  0.15433155,
        -0.12978497,  0.04102806],
       [-0.12802195, -0.01248195, -0.40373683,  0.16524761, -0.21355242,
        -0.11611039,  0.17217344],
       [ 0.10901868,  0.10742295, -0.33991777,  0.34568402,  0.41579154,
        -0.41346813, -0.06208598],
       [ 0.15044401,  0.30194746,  0.16487676,  0.00236012,  0.22801632,
        -0.33234616, -0.39889591],
       [ 0.19377317,  0.18757675,  0.21732211, -0.25059196,  0.17593481,
        -0.2548917 ,  0.21677275],
       [-0.26099772, -0.18226862,  0.30622801,  0.48832368, -0.25368178,
        -0.09021058,  0.33618684],
       [ 0.4245586 ,  0.21215213,  0.3248986 ,  0.11938699,  0.21710144,
        -0.10805605, -0.00250705],
       [ 0.22111872,


100%|██████████| 32/32 [1:26:41<00:00, 162.53s/it]   

[(928, (array([[-0.00983509,  0.37644497,  0.11783056, -0.29725424,  0.36965628,
         0.19557471,  0.2649842 ],
       [-0.34438596,  0.07054506, -0.21055911, -0.01749544,  0.24620724,
         0.09422955,  0.3938419 ],
       [ 0.02952526, -0.11749438, -0.42460562, -0.29875602,  0.42265305,
         0.38456951,  0.37730809],
       [-0.24829754,  0.45614862,  0.28979568,  0.48564381, -0.38356555,
        -0.08076501, -0.16016484],
       [ 0.49576358,  0.16457599,  0.27315264, -0.13538901, -0.28272893,
        -0.48452731,  0.02153893],
       [ 0.15792776,  0.22634476, -0.41784706, -0.07683516,  0.13327813,
         0.14297219, -0.39805172],
       [ 0.14799946,  0.22251032,  0.40895599,  0.31778757,  0.22274575,
        -0.22909378, -0.08972052],
       [-0.23491185,  0.04078511,  0.13528503,  0.07952469, -0.31922162,
        -0.27090597,  0.25348003],
       [ 0.04081968,  0.13374796,  0.34200511,  0.42029785,  0.06821344,
        -0.0088222 ,  0.44597875],
       [-0.33488403,

In [12]:
from numpy.random import SeedSequence, default_rng
ss = SeedSequence(12345678)

# Spawn off 32 child SeedSequences to pass to child processes.
child_seeds = ss.spawn(20)
streams = [default_rng(s) for s in child_seeds]

In [13]:
gridsize=20
Strike=1.0

yvalues=np.zeros([gridsize,7]);

for x in range(0,gridsize,1):
    yvalues[x,0]=x/gridsize-0.5;
    for k in range(21,7,1):
        yvalues[x,k]=0;

In [14]:
def magic_function(f):
    Strike = 1.0
    n=100000
    TestSizePerNode = 1 #312500
    StartingValuesTestsetLocal= f[2]

    IncrementsBM = f[1].normal(0,1,size=[1,n,7,L])
    MCprices=np.zeros(TestSizePerNode)

    price=0
    for incBM in range(0,n,1):    
        tmpvec=evalCurve(StartingValuesTestsetLocal,IncrementsBM[0,incBM],taugrid)
        tmp=np.sum(tmpvec)*(12/365)
        price+=max(tmp-Strike,0)
    MCprices[0]=price/n
    return StartingValuesTestsetLocal, MCprices


def process_frame(f):
    return f[0], magic_function(f)





In [15]:
from tqdm import tqdm

from multiprocess import Pool
#from magic_functions import process_frame

frames_list = [[i,streams[i],yvalues[i]] for i in range(0,len(streams),1)]

max_pool = 32

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(process_frame,
                   frames_list),
            total=len(frames_list)
        )
    )    

print(pool_outputs)
new_dict = dict(pool_outputs)





100%|██████████| 20/20 [08:05<00:00, 24.29s/it]  

[(0, (array([-0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.00353111]))), (1, (array([-0.45,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.00529816]))), (2, (array([-0.4,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.0084364]))), (3, (array([-0.35,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.01232508]))), (4, (array([-0.3,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.01752013]))), (5, (array([-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.02472338]))), (6, (array([-0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.03467203]))), (7, (array([-0.15,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.04771445]))), (8, (array([-0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.06423747]))), (9, (array([-0.05,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.08333268]))), (10, (array([0., 0., 0., 0., 0., 0., 0.]), array([0.10868996]))), (11, (array([0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]), array([0.13734527]))), (12, (array([0.1, 0. , 0. , 0

In [16]:
TestSizePerNode=1
TotalTestSize=TestSizePerNode*len(streams)
startingValues=np.zeros((TotalTestSize,7))
MCprices=np.zeros(TotalTestSize)
for x in range(0, len(streams), 1):
    startingValues[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][0]
    MCprices[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][1]
print(startingValues)
print(MCprices)
np.save('MCprices1',MCprices)
np.save('xValuesE1', yvalues)

[[-0.5   0.    0.    0.    0.    0.    0.  ]
 [-0.45  0.    0.    0.    0.    0.    0.  ]
 [-0.4   0.    0.    0.    0.    0.    0.  ]
 [-0.35  0.    0.    0.    0.    0.    0.  ]
 [-0.3   0.    0.    0.    0.    0.    0.  ]
 [-0.25  0.    0.    0.    0.    0.    0.  ]
 [-0.2   0.    0.    0.    0.    0.    0.  ]
 [-0.15  0.    0.    0.    0.    0.    0.  ]
 [-0.1   0.    0.    0.    0.    0.    0.  ]
 [-0.05  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.05  0.    0.    0.    0.    0.    0.  ]
 [ 0.1   0.    0.    0.    0.    0.    0.  ]
 [ 0.15  0.    0.    0.    0.    0.    0.  ]
 [ 0.2   0.    0.    0.    0.    0.    0.  ]
 [ 0.25  0.    0.    0.    0.    0.    0.  ]
 [ 0.3   0.    0.    0.    0.    0.    0.  ]
 [ 0.35  0.    0.    0.    0.    0.    0.  ]
 [ 0.4   0.    0.    0.    0.    0.    0.  ]
 [ 0.45  0.    0.    0.    0.    0.    0.  ]]
[0.00353111 0.00529816 0.0084364  0.01232508 0.01752013 0.02472338
 0.03467203 0.04771445 0.0642374

In [17]:
gridsize=20
Strike=1.0

yvalues=np.zeros([gridsize,7]);

for x in range(0,gridsize,1):
    yvalues[x,0]=0.;
    yvalues[x,1]=x/gridsize-0.5;
    for k in range(2,7,1):
        yvalues[x,k]=0;

In [18]:
def magic_function(f):
    Strike = 1.0
    n=100000
    TestSizePerNode = 1 #312500
    StartingValuesTestsetLocal= f[2]

    IncrementsBM = f[1].normal(0,1,size=[1,n,7,L])
    MCprices=np.zeros(TestSizePerNode)

    price=0
    for incBM in range(0,n,1):    
        tmpvec=evalCurve(StartingValuesTestsetLocal,IncrementsBM[0,incBM],taugrid)
        tmp=np.sum(tmpvec)*(12/365)
        price+=max(tmp-Strike,0)
    MCprices[0]=price/n
    return StartingValuesTestsetLocal, MCprices


def process_frame(f):
    return f[0], magic_function(f)




In [19]:
from tqdm import tqdm

from multiprocess import Pool
#from magic_functions import process_frame

frames_list = [[i,streams[i],yvalues[i]] for i in range(0,len(streams),1)]

max_pool = 32

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(process_frame,
                   frames_list),
            total=len(frames_list)
        )
    )    

print(pool_outputs)
new_dict = dict(pool_outputs)




100%|██████████| 20/20 [08:15<00:00, 24.78s/it]  

[(0, (array([ 0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.14233128]))), (1, (array([ 0.  , -0.45,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.13916518]))), (2, (array([ 0. , -0.4,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.13553241]))), (3, (array([ 0.  , -0.35,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.13143004]))), (4, (array([ 0. , -0.3,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.12734173]))), (5, (array([ 0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.12320437]))), (6, (array([ 0. , -0.2,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.12054828]))), (7, (array([ 0.  , -0.15,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.11767361]))), (8, (array([ 0. , -0.1,  0. ,  0. ,  0. ,  0. ,  0. ]), array([0.11485926]))), (9, (array([ 0.  , -0.05,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]), array([0.11064919]))), (10, (array([0., 0., 0., 0., 0., 0., 0.]), array([0.10868996]))), (11, (array([0.  , 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ]), array([0.10518411]))), (12, (array([0. , 0.1, 0. , 

In [20]:
TestSizePerNode=1
TotalTestSize=TestSizePerNode*len(streams)
startingValues=np.zeros((TotalTestSize,7))
MCprices=np.zeros(TotalTestSize)
for x in range(0, len(streams), 1):
    startingValues[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][0]
    MCprices[x*TestSizePerNode:(x+1)*TestSizePerNode]=pool_outputs[x][1][1]
print(startingValues)
print(MCprices)
np.save('MCprices2',MCprices)
np.save('xValuesE2', yvalues)

[[ 0.   -0.5   0.    0.    0.    0.    0.  ]
 [ 0.   -0.45  0.    0.    0.    0.    0.  ]
 [ 0.   -0.4   0.    0.    0.    0.    0.  ]
 [ 0.   -0.35  0.    0.    0.    0.    0.  ]
 [ 0.   -0.3   0.    0.    0.    0.    0.  ]
 [ 0.   -0.25  0.    0.    0.    0.    0.  ]
 [ 0.   -0.2   0.    0.    0.    0.    0.  ]
 [ 0.   -0.15  0.    0.    0.    0.    0.  ]
 [ 0.   -0.1   0.    0.    0.    0.    0.  ]
 [ 0.   -0.05  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.05  0.    0.    0.    0.    0.  ]
 [ 0.    0.1   0.    0.    0.    0.    0.  ]
 [ 0.    0.15  0.    0.    0.    0.    0.  ]
 [ 0.    0.2   0.    0.    0.    0.    0.  ]
 [ 0.    0.25  0.    0.    0.    0.    0.  ]
 [ 0.    0.3   0.    0.    0.    0.    0.  ]
 [ 0.    0.35  0.    0.    0.    0.    0.  ]
 [ 0.    0.4   0.    0.    0.    0.    0.  ]
 [ 0.    0.45  0.    0.    0.    0.    0.  ]]
[0.14233128 0.13916518 0.13553241 0.13143004 0.12734173 0.12320437
 0.12054828 0.11767361 0.1148592

In [9]:
import time

gridsize=20
Strike=1.0

n=50000
yvalues=np.zeros([gridsize,7]);

for x in range(0,gridsize,1):
    yvalues[x,0]=0.;
    yvalues[x,1]=x/gridsize-0.5;
    for k in range(2,7,1):
        yvalues[x,k]=0;

#testinput=tf.reshape(tf.constant(yvalues),[gridsize,10])
#prediction=model(testinput)
tic = time.perf_counter()

IncrementsBM = np.random.normal(0,1,size=[gridsize,n,7,L])
MCprices=np.zeros(gridsize)
for x in range(0,gridsize,1):
    price=0
    for incBM in range(0,n,1):    
        tmpvec=evalCurve(yvalues[x,:],IncrementsBM[x,incBM],taugrid)
        tmp=np.sum(tmpvec)*(12/365)
        price+=max(tmp-Strike,0)
    MCprices[x]=price/n

toc = time.perf_counter()
print(f"Calc time: {toc - tic:0.4f} seconds")
np.save('MCprices2',MCprices)
np.save('xValuesE2', yvalues)
print(MCprices)

Calc time: 3111.2664 seconds
[0.14301747 0.14096418 0.13426895 0.13110012 0.12640977 0.12517613
 0.122051   0.11649869 0.11540776 0.11083141 0.10787003 0.1057147
 0.10207486 0.10038773 0.0984001  0.09382389 0.09081068 0.08787938
 0.08538164 0.08281219]
